Одним из главных плюсов больших языковых моделей является возможность использовать их без дополнительного обучения. Зачастую решить необходимую задачу можно просто с помощью хорошо сформулированной инструкции. При наличии 10-50 примеров можно добавить их в инструкцию, чтобы лучше "объяснить", в чем состоит задача.

<center><img src="https://i.postimg.cc/Hn8zmRV7/prompt-engineering.png" width="800"></center>

Однако может случиться, что задача слишком сложная и требует "знаний", которые не встречались при предобучении LLM. Другой пример — применение к языку, данные на котором отсутствовали в обучающем датасете.

Тогда можно осуществить тоннкую настройку большой языковой модели (аналогично энкодерной модели). Важно, что для этого необходимо достаточное количество примеров.

Проблема состоит в том, что дообучение всей LLM требует слишком много времени и ресурсов. Существуют различные методы, которые позволяют осуществлять тонкую настройку более эффективно и менять только часть весов модели.

<center><img src="https://i.postimg.cc/XNBt4zM8/Parameter-efficient-Fine-tuning.png" width="800"></center>

# Parameter-Efficient Fine-Tuning (PEFT)

PEFT — это методы тонкой настройки, которые позволяют улучшить результаты работы языковых моделей при выполнении определенных задач. Их идея заключается в том, чтобы обучить небольшое подмножество параметров предобученной LLM, оставляя бо́льшую часть замороженными. Благодаря этому мы также можем избежать переобучения на новых данных.

## Promt tuning

[The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/abs/2104.08691)

Первый метод PEFT в некотором смысле является развитием идеи prompt engineering.

Подбор инструкции вручную состоит в том, чтобы эмбеддинги входящих к нее слов при подаче в модель привели к желаемому результату генерации. Идея promt tuning состоит в том, чтобы избавиться от шага с подбором слов и сразу подбирать нужные эмбеддинги.

Фиксированные инструкции (hard prompt) подбираются человеком и включают токены из словаря. Токены нефиксированной инструкции (soft prompt) являются обучаемыми векторами.

<center><img src="https://i.postimg.cc/SxGxVKDJ/hard-soft.jpg" width="700"></center>

Перед входом в языковую модель размещается несколько обучаемых эмбеддингов.
- У модели есть эмбеддинги токенов (input text), которые привязаны к осмысленным словам.
- Также есть эмбеддинги, которые не зависят от слов, просто обучаемые векторы, причем каждый из них уникальный.

<center><img src="https://i.postimg.cc/x8VwWvdY/image1.png" width="750"></center>

Чтобы создать soft prompt для некоторой задачи, сначала инициализируется промпт в виде последовательности векторов фиксированной длины (например, длиной в 20 токенов). Эти векторы объединяются с входными примерами и подаются в модель.

Предсказание модели сравнивается с целевым значением для расчета функции потерь, и её значение передается обратно для расчета градиентов. Однако обновляться будут только новые обучаемые векторы, тогда как базовая модель остается замороженной.

Soft prompts, полученные таким образом, не поддаются человеческой интерпретации. На интуитивном уровне они содержат информацию о том, как наилучшим образом сформулировать задачу, имея размеченный набор данных. Они выполняют ту же роль, что и написанные вручную инструкции, но без необходимости сопоставлять эмбеддингам слова человеческого языка.

Чем больше размер модели, тем лучше работает данный метод.

<center><img src="https://i.postimg.cc/SKNCf37b/image3.png" width="400"></center>

## Prefix tuning

[Prefix-Tuning: Optimizing Continuous Prompts for Generation](https://arxiv.org/abs/2101.00190)

[P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/abs/2110.07602)

Развитие идеи promt tuning: вместо обучения промпта для входного слоя обучать свои промпты для каждого слоя.

Количество обучаемых параметров увеличивается в N раз, где N — количество слоев.

<em>Prompt tuning</em>
<center><img src ="https://i.postimg.cc/cHFDcf8X/prompt.png" width="800"></center>

<em>Prefix tuning</em>
<center><img src ="https://i.postimg.cc/zf50ZxCx/prefix.png" width="800"></center>

## Адаптеры

[Parameter-Efficient Transfer Learning for NLP](https://arxiv.org/abs/1902.00751)

Вместо того чтобы обучать эмбеддинги дополнительных фиктивных токенов, предлагается добавить в архитектуру нейросети маленькие обучаемые слои — адаптеры.

Добавляя их после слоев внимания (multi-head attention) и линейных слоев (feed-forward) в архитектуре Трансформер, мы можем обновлять только веса в адаптерах во время тонкой настройки, сохраняя при этом остальные параметры модели неизменными.

Адаптер содержит два линейных слоя, между которыми находится нелинейная функция активации. Входными данными для слоя адаптера будет скрытое представление $h$, которое является выходом слоя множественного внимания.

<center><img src ="https://i.postimg.cc/ZnF6cSgZ/adapter.png" width="800"></center>

При проходе через первый линейный слой адаптера вектор $h$ проецируется в пространство малой размерности. После этого применяется функция активации. Выход второго линейного слоя имеет ту же размерность, как у изначального вектора $h$. Вектор $\Delta h$, полученный при проходе через адаптор, суммируется с исходным вектором $h$ (skip-connection).

## LoRA

[LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)

Существуют адаптеры, которые работают на уровне трансформерного слоя, а на уровне весов. Идея состоит в том, чтобы модифицировать каждую матрицу слоя множественного внимания: `q_proj` (query), `k_proj` (key), `v_proj` (value).

<center><img src ="https://i.postimg.cc/7PFx1zMx/Transformer-architecture-in-wav2vec2-along-with-Lo-RA.png" width="800"></center>

Чтобы подробнее разобрать метод LoRA, спустимся на более простой уровень. Пусть у нас есть один линейный слой без функции активации.

Если на вход подадим $x$, на выходе получим $y = Wx$ где $W$ — матрица весов.

Мы хотим немного изменить принцип работы этого слоя, дообучив модель, скорректировав веса на $\Delta W$ (которые ищут обычным градиентным спуском), так что бы новый выход был:

$$y=W'x=(W+\Delta W)x=y+\Delta Wx$$

<center><img src ="https://i.postimg.cc/SRkHVcdP/48f5024475f644b2d1cacbda2a8cb0b6.png" width="300"></center>

Таким образом, мы можем зафиксировать веса матрицы $W$, а вместо этого учить $\Delta W$ — матрицу, предсказывающую отличие результата обычной модели, от дообученой.

Сразу возникнет вопрос — а где тут выигрыш? Ведь размеры матриц $W$ и $\Delta W$ должны быть одинаковыми, так что в них одинаковое количество обучаемых параметров.

Вот тут и включаются в игру слова Low Rank — матрицу маленького ранга можно представить как произведение двух меньшей размерности. Наша матрица может быть размером 100 на 70, но ранг, то есть количество линейно независимых строк или столбцов (таких столбцов которые действительно содержат новую информацию о модели, а не действуют на вектор параметров аналогично соседям) может быть меньше, чем 70, — например 4 или 20.

Рассмотрим ненулевой вектор  $\vec a=(1, -1, 2)$. Его ранг равен 1.

Рассмотрим матрицу, строки которой линейно зависимы (выражаются друг через друга).

$$\begin{pmatrix}
1 & -1 & 2\\
2 & -2 & 4
\end{pmatrix}$$

С геометрической точки зрения во вторую строку записаны координаты коллинеарного вектора $2 \vec a=(2, -2, 4)$. Таким образом, ранг данной матрицы тоже равен 1.

Познакомимся с матрицей, строки которой линейно независимы.

$$\begin{pmatrix}
1 & -1 & 2\\
0 & 1 & -1
\end{pmatrix}$$

Пара векторов $\vec a=(1, -1, 2)$ и $\vec b=(0, 1, -1)$ не коллинеарны. Ран матрицы равен 2.

Ранг матрицы – это максимальное количество линейно независимых строк. Или: ранг матрицы – это максимальное количество линейно независимых столбцов. Их количество всегда совпадает.

Из вышесказанного также следует важный практический ориентир: ранг матрицы не превосходит её минимальной размерности.

Таким образом, можно представить матрицу $\Delta W$ как произведение двух матриц $A$ и $B$. При этом сильно выиграем в количестве обучаемых параметров.

Для примера на картинке матрица 100 х 70 содержит 7000 чисел, а две в левой части неравенства 140 + 200 = 340.

<center><img src ="https://i.postimg.cc/25WJD0XJ/79d036c365cd35a10ae1c80cc3e5a2e1.png" width="300"></center>



В общем случае потребуется обучать в $\frac{ nr + rn }{ n^2 } = \frac{ 2r }{n}$ меньше параметров. $r$ выбирается маленьким, порядка 2-8, что делает это значение очень маленьким $\approx 10^{-2}$.

Мы немного потеряем в общности, так как теперь  автоматический постулируем, что у $\Delta W$ низкий ранг. Однако в этом нет ничего страшного: разработчики LoRA утверждают что хотя LLM имеют миллионы или даже миллиарды параметров, они имеют низкую "внутреннюю размерность" (intrinsic dimension) при адаптации к новой задаче. Проще говоря, большинство параметров являются избыточными. Из чего можно сделать вывод, что матрицы можно представить пространством меньшей размерности, сохраняя при этом большую часть важной информации.

[Intrinsic Dimensionality Explains the Effectiveness of Language Model Fine-Tuning](https://arxiv.org/abs/2012.13255)

Таким образом, во время обучения нам необходимо хранить в памяти веса $W$ исходной модели и $\Delta W=B\cdot A$ дообучаемой, а считать градиенты только для "новых" маленьких матриц $A$ и $B$.

При инициализации модели мы создаем матрицу $B$ случайным образом, а матрицу $A$ инициализируем нулями, что бы изначально $\Delta W = 0$.

<center><img src ="https://i.postimg.cc/vHD0yP4z/252844b9dbfa3f1125a54997087dd2f5.png" width="300"></center>

# Тонкая настройка LLM с помощью LoRA

Мы рассмотрим, как осуществить тонкую настройку больших языковых моделей с помощью метода LoRa (Low-Rank Adaptation), который:
- Замораживает значения весов для предобученной модели
- Добавляет небольшие обучаемые матрицы ранговой декомпозиции к слоям внимания
- Обычно уменьшает количество обучаемых параметров примерно на 90%
- Поддерживает производительность модели при сохранении эффективной работы с памятью

In [ ]:
# Установка необходимых библиотек
!pip install transformers datasets trl huggingface_hub -q

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from trl import SFTConfig, SFTTrainer, setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Получаем токен из Secrets Colab
hf_token = userdata.get('hf_token')

# Авторизуемся в Hugging Face Hub
login(token=hf_token)

Класс [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) из библиотеки `trl` обеспечивает интеграцию с адаптерами LoRa через библиотеку [PEFT](https://huggingface.co/docs/peft/en/index).

Основные преимущества включают в себя:

**1. Эффективность использования памяти:**
- В памяти графического процессора хранятся только параметры адаптера
- Веса базовой модели остаются замороженными и могут быть загружены с меньшей точностью
- Позволяет выполнять тонкую настройку больших моделей на потребительских графических процессорах

**2. Особенности обучения:**
- Встроенная интеграция PEFT/LoRa с минимальными настройками
- Поддержка QLoRA (Quantized LoRa) для еще большей эффективности использования памяти

**3. Управление адаптером:**
- Сохранение весов адаптера во время контрольных точек (checkpoints)
- Функции для присоединения адаптеров обратно к базовой модели

Для тонкой настройки требуется всего несколько шагов:
- Определить конфигурацию LoRa (rank, alpha, dropout)
- Создать SFTTrainer с помощью PEFT config
- Обучить и сохранить веса адаптера

## Загрузка данных и базовой модели

Загрузим датасет, который будем использовать для тонкой настройки.

In [ ]:
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

In [ ]:
dataset["train"][0]

Определим предобученную модель и токенизатор.

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и токенизаторы
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Определим шаблон чата
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Зададим директорию, куда будут сохраняться веса адаптора
finetune_name = "SmolLM2-FT-MyDataset"

Протестируем предобученную модель для генерации.

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

In [ ]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]

In [ ]:
def test_inference(prompt, pipe):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()

In [ ]:
for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt, pipe)}")
    print("-" * 50)

## Тонкая настройка

`SFTTrainer` поддерживает интеграцию с библиотекой `peft`, что упрощает эффективную настройку LLM, например, с использованием LoRa. Нам нужно только создать `LoraConfig` и передать его в `trainer`.

Определим параметры LoRA.

In [ ]:
# r: измерение ранга для матриц обновления LoRa (меньшее значение = большее сжатие)
rank_dimension = 6
# lora_alpha: коэффициент масштабирования для слоев LoRa (чем выше, тем больше влияние на исходные веса)
lora_alpha = 8
# lora_dropout: вероятность зануления слоев LoRa (помогает предотвратить переобучение)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # ранговый показатель - обычно от 4 до 32
    lora_alpha=lora_alpha,  # коэффициент масштабирования LoRa - обычно 2х для ранга
    lora_dropout=lora_dropout,  # вероятность зануления для слоев LoRa
    target_modules="all-linear",  # к каким модулям применять LoRa
    task_type="CAUSAL_LM",  # тип задачи для архитектуры модели
)

Прежде чем начать обучение, нам нужно определить гиперпараметры (`TrainingArguments`), которые будем использовать.

In [ ]:
args = SFTConfig(
    # Параметры сохранения
    output_dir=finetune_name,
    # Продолжительность обучения
    num_train_epochs=1,
    # Настройки размера батча
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    # Оптимизация памяти
    gradient_checkpointing=True,
    # Настройки оптимизатора
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    # Планировщик скорости обучения
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    # Логирование и сохранение
    logging_steps=10,
    save_strategy="epoch",
    # Настройки точности
    bf16=True,
    # Настройки интеграции
    push_to_hub=False,
    report_to="none",
)

Теперь есть все необходимые компоненты для создания `SFTTrainer`, чтобы начать тонкую настройку модели.

In [ ]:
# Создаем SFTTrainer с конфигурацией LoRa
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    processing_class=tokenizer
    )

Начнем обучение модели, вызвав метод train() для экземпляра класса Trainer. Поскольку мы используем метод PEFT, сохранятся только адаптированные веса модели, а не полная модель.

In [ ]:
# после начала обучения модель будет сохранена в выходной директории
trainer.train()
trainer.save_model()

## Объединение адаптора LoRA с предобученной моделью

При использовании LoRa мы обучаем только веса адаптеров, сохраняя базовую модель замороженной. Во время обучения мы сохраняем только эти веса адаптеров (~2-10 МБ), а не полную копию модели. Однако для развертывания может потребоваться объединить адаптеры обратно в базовую модель.

In [ ]:
# загрузка весов адаптора на CPU
lora_model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# объединение LoRA с базовой моделью и сохранение
merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained(args.output_dir,
                             safe_serialization=True,
                             max_shard_size="2GB")

Протестируем несколько примеров и посмотрим, как работает модель после тонкой настройки.

In [ ]:
merged_pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, device=device)

In [ ]:
for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt, merged_pipe)}")
    print("-" * 50)

## Анализ изменений для конкретного слоя

Исходная модель имеет следующую архитектуру.

In [ ]:
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)
model

Возьмем матрицу весов Q (query) для первого слоя.

In [ ]:
model.model.layers[0].self_attn.q_proj

In [ ]:
base = model.model.layers[0].self_attn.q_proj.weight.data
base

Убедимся, что модель после тонкой настройки имеет аналогичную структуру.

In [ ]:
merged_model

Возьмем ту же самую матрицу весов и убедимся, что она отличается.

In [ ]:
merged_model.model.layers[0].self_attn.q_proj

In [ ]:
merged = merged_model.model.layers[0].self_attn.q_proj.weight.data
merged

Запишем в отдельные переменные низкоранговые матрицы A и B, которые обучались с помощью метода LoRA.

In [ ]:
trainer.model.base_model.model.model.layers[0].self_attn.q_proj.lora_A.default

In [ ]:
a = trainer.model.base_model.model.model.layers[0].self_attn.q_proj.lora_A.default.weight.data
a

In [ ]:
trainer.model.base_model.model.model.layers[0].self_attn.q_proj.lora_B.default

In [ ]:
b = trainer.model.base_model.model.model.layers[0].self_attn.q_proj.lora_B.default.weight.data
b

Убедимся, что при их перемножении и прибавлении к матрице базовой модели получим такую же матрицу, как в модели после тонкой настройки.

<center><img src ="https://i.postimg.cc/438KX176/eddf88ff00409fa19ff21826a26e90d2.gif" width="600"></center>

In [ ]:
base + lora_alpha/rank_dimension * (b @ a)

In [ ]:
merged